## Распознавание Именованных Сущностей с помощью библиотеки DeepPavlov
---

<img align="center" height="27%" width="27%" src="https://deeppavlov.ai/docs/_static/ipavlov_logo.png"/>


Задачей Распознавания Именованных Сущностей (NER) называется извлечение из текста таких объектов как 
**имена**, **названия организаций**, **названия географических объектов**. Данная задача как правило является компонентом в более крупной системе. Например, в диалоговой системе NER может быть использован для выделения имени собеседника. В библиотеке [DeepPavlov](https://github.com/deepmipt/DeepPavlov) есть ряд моделей которые решают данную задачу. Мы рассмотрим две модели решающие задачу NER на русском языке: [BERT](https://arxiv.org/pdf/1810.04805.pdf), показывающий на данный момент наилучшее качество, и [Bi-LSTM-CRF](https://arxiv.org/pdf/1603.01360.pdf) - уступающий по метрикам, однако более быстрый baseline.

[NER Demo](https://demo.ipavlov.ai/#/ru/ner)

## Постановка задачи

Задача НЕР может быть поставлена следующим образом: для заданной последовательность слов предсказать последовательность меток. Каждому входному токену сопоставляется метка из заданного множества меток. Пример: 

    Microsoft объявила о приобретении Skype Technologies
     ORG         O     O       O       ORG      ORG

здесь **PER** - персона, **LOC** - локация. Однако, представленная разметка не позволяет разделять подряд идущие сущности. Для разделения таких сущностей используют префиксы B и I перед каждой меткой кроме O. Префикс B обозначает начало сущности, а I - продолжение. Тогда для примера выше будет следующая разметка:

    Microsoft объявила о приобретении Skype Technologies
      B-ORG      O     O       O      B-ORG    I-ORG

Разметка с префиксами B и O - наиболее распространённый способ разметки данных. Данный тип разметки часто называют **BIO** или **IOB**.

## Dataset

Рассматриваемые в данном notebook-е модели были обучены на датасете [1]. Данный датасет содержит 1000 новостей в которых размечены персоны (PER), локации (LOC) и организации (ORG). В силу того, что обучающая выборка содержит только новостные данные смена типов распознаваемых текстов может существенно отразиться на качестве работы системы. Например, при использовании модели обученной на новостях переход к распознавания диалогов службы поддрежки может существенно снизить качество работы системы. 

1. Mozharova V., Loukachevitch N., Two-stage approach in Russian named entity recognition // International FRUCT Conference on Intelligence, Social Media and Web, ISMW FRUCT 2016. Saint-Petersburg; Russian Federation, DOI 10.1109/FRUCT.2016.7584769 

## Установка библиотеки

In [ ]:
!pip install -q deeppavlov

## Установка зависимостей

In [ ]:
!python -m deeppavlov install ner_rus
!python -m deeppavlov install ner_rus_bert

## Использование моделей 

### BERT 

BERT - сеть архитектуры Transformer предобученная на задаче Masked Language Modelling (MLM). Модель осуществляющая РИС использует [RuBERT](https://arxiv.org/abs/1905.07213) предобученный на русском языке. 

In [ ]:
from deeppavlov import configs, build_model

config_path = configs.ner.ner_rus_bert

ner = build_model(config_path, download=True)

In [ ]:
sentence = 'Директором ОАО "Страна Чудес" назначена Алиса'

# Можно так же список слов
# sentence = ['Директором', 'ОАО', '"', 'Страна', 'Чудес', '"', 'назначена', 'Алиса']

tokens, tags = ner(['Директором ОАО "Страна Чудес" назначена Алиса'])
for tok, tag in zip(tokens[0], tags[0]):
    print(f'{tok}\t{tag}')

### Bi-LSTM-CRF
Архитектура [Bi-LSTM-CRF](https://arxiv.org/pdf/1603.01360.pdf) проще BERT как по памяти, так и по сложности вычисления. Данная архитектура уступает в качетсве BERT, однако, работает заметно быстрее.

In [5]:
ner = build_model(configs.ner.ner_rus, download=True)

2019-08-17 20:42:10.930 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_rus_v3_cpu_compatible.tar.gz to /home/com/.deeppavlov/ner_rus_v3_cpu_compatible.tar.gz
100%|██████████| 5.88M/5.88M [00:00<00:00, 11.9MB/s]
2019-08-17 20:42:11.465 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting /home/com/.deeppavlov/ner_rus_v3_cpu_compatible.tar.gz archive into /home/com/.deeppavlov/models
2019-08-17 20:42:11.596 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/embeddings/lenta_lower_100.bin to /home/com/.deeppavlov/downloads/embeddings/lenta_lower_100.bin
100%|██████████| 1.11G/1.11G [01:36<00:00, 11.5MB/s]
2019-08-17 20:43:48.856 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /home/com/.deeppavlov/models/ner_rus/word.dict]
2019-08-17 20:43:48.918 INFO in 'deeppavlov.core.data.simple_vocab'['simple_v

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2019-08-17 20:43:59.228 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
/home/com/miniconda2/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-08-17 20:44:01.370 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 50: [loading model from /home/com/.deeppavlov/models/ner_rus/model]


INFO:tensorflow:Restoring parameters from /home/com/.deeppavlov/models/ner_rus/model


In [6]:
tokens, tags = ner(['Директором ОАО "Страна Чудес" назначена Алиса'])
for tok, tag in zip(tokens[0], tags[0]):
    print(f'{tok}\t{tag}')

Директором	O
ОАО	B-ORG
``	O
Страна	B-LOC
Чудес	I-LOC
''	O
назначена	O
Алиса	B-PER


## DeepPavlov documentation: 

### http://docs.deeppavlov.ai/en/master/components/ner.html

## GitHub Repository

### https://github.com/deepmipt/DeepPavlov

## DeepPavlov demo page

### https://demo.ipavlov.ai

## Forum

### https://forum.ipavlov.ai
